# Tutorial: Agentic retrieval using Azure AI Search and Foundry Agent Service

Use this notebook to create an agentic retrieval pipeline built on Azure AI Search and Foundry Agent Service.

In this notebook, you:

1. Create and load an `earth-at-night` search index.

1. Create an `earth-knowledge-source` that targets your index.

1. Create an `earth-knowledge-base` that targets your knowledge source and an LLM for intelligent query planning.

1. Use the knowledge base to fetch, rank, and synthesize relevant information from the index.

1. Create an agent in Foundry Agent Service to determine when queries are needed.

1. Create an MCP tool to orchestrate all requests.

1. Start a chat with the agent.

This notebook is referenced in [Tutorial: Build an end-to-end agentic retrieval solution using Azure AI Search](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-pipeline).

Unlike [Quickstart: Use agentic retrieval in Azure AI Search](https://learn.microsoft.com/azure/search/search-get-started-agentic-retrieval), this quickstart uses Foundry Agent Service to determine whether to retrieve data from the knowledge source and uses an MCP tool for orchestration.

## Prerequisites

+ An Azure AI Search service in any [region that provides agentic retrieval](https://learn.microsoft.com/azure/search/search-region-support).

+ A [Microsoft Foundry project](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/create-projects?view=foundry-classic&tabs=foundry) and resource. When you create a project, the resource is automatically created.

+ A [supported LLM](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models) deployed to your project. This notebook uses `gpt-5-mini`. We recommend a minimum token capacity of 100,000. You can find the LLM's capacity and rate limit in the Foundry portal. If you want vectorization at query time, you should also deploy a text embedding model.

+ [Visual Studio Code](https://code.visualstudio.com/download) with the [Python extension](https://marketplace.visualstudio.com/items?itemName=ms-python.python) and [Jupyter package](https://pypi.org/project/jupyter/).

## Configure access

This notebook assumes that you're using Microsoft Entra ID for authentication and role assignments for authorization.

To configure role-based access:

1. Sign in to the [Azure portal](https://portal.azure.com).

1. On your Azure AI Search service:

    1. [Enable role-based access](https://learn.microsoft.com/azure/search/search-security-enable-roles).
    
    1. [Create a system-assigned managed identity](https://learn.microsoft.com/azure/search/search-howto-managed-identities-data-sources#create-a-system-managed-identity).
    
    1. [Assign the following roles](https://learn.microsoft.com/azure/search/search-security-rbac#how-to-assign-roles-in-the-azure-portal) to yourself.
    
       + **Search Service Contributor**
    
       + **Search Index Data Contributor**
    
       + **Search Index Data Reader**

    1. Assign **Search Index Data Reader** to your Microsoft Foundry project.

1. On your Microsoft Foundry resource:

    1. Assign the following roles to yourself.

        + **Azure AI User**

        + **Azure AI Project Manager**

    1. Assign **Cognitive Services User** to the managed identity of your search service.

## Set up connections

Save the `sample.env` file as `.env` and then modify the environment variables to use your Azure endpoints. You need endpoints for:

+ Azure AI Search
+ Azure OpenAI (for the models deployed to your project)
+ Microsoft Foundry project

You also need the resource ID of your project. You can find all of these values in the [Azure portal](https://portal.azure.com/).

## Load connections

We recommend creating a virtual environment to run this sample code. In Visual Studio Code, open the control palette (ctrl-shift-p) to create an environment. This notebook was tested on Python 3.13.7.

After your environment is created, load the environment variables to set up connections and object names.

In [ ]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.mgmt.core.tools import parse_resource_id
import os

load_dotenv(override=True) # take environment variables from .env.

project_endpoint = os.environ["PROJECT_ENDPOINT"]
project_resource_id = os.environ["PROJECT_RESOURCE_ID"]
project_connection_name = os.getenv("PROJECT_CONNECTION_NAME", "earthknowledgeconnection")
agent_model = os.getenv("AGENT_MODEL", "gpt-4.1-mini")
agent_name = os.getenv("AGENT_NAME", "earth-knowledge-agent")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
knowledge_source_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_SOURCE_NAME", "earth-knowledge-source")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth-at-night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
base_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-knowledge-base")

# Parse the resource ID to extract subscription and other components
parsed_resource_id = parse_resource_id(project_resource_id)
subscription_id = parsed_resource_id['subscription']
resource_group = parsed_resource_id['resource_group']
account_name = parsed_resource_id['name']
project_name = parsed_resource_id['child_name_1']

## Create a search index

This steps create a search index that contains plain text and vector content. You can use an existing index, but it must meet the [criteria for agentic retrieval workloads](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-index). The primary schema requirement is a semantic configuration with a `default_configuration_name`.

In [ ]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient

index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")

Index 'earth-at-night' created or updated successfully


## Upload sample documents

This notebook uses data from NASA's Earth at Night e-book. The data is retrieved from the [azure-search-sample-data](https://github.com/Azure-Samples/azure-search-sample-data) repository on GitHub and passed to the search client for indexing.

In [ ]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}'")

Documents uploaded to index 'earth-at-night'


## Create a knowledge source

This step creates a knowledge source that targets the index you previously created. In the next step, you create a knowledge base that uses the knowledge source to orchestrate agentic retrieval.


In [4]:
from azure.search.documents.indexes.models import SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters, SearchIndexFieldReference
from azure.search.documents.indexes import SearchIndexClient

ks = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Knowledge source for Earth at night data",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        source_data_fields=[SearchIndexFieldReference(name="id"), SearchIndexFieldReference(name="page_number")]
    ),
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{knowledge_source_name}' created or updated successfully.")

Knowledge source 'earth-knowledge-source' created or updated successfully.


## Create a knowledge base

This step creates a knowledge base, which acts as a wrapper for your knowledge source and LLM deployment.

`EXTRACTIVE_DATA` is the default modality and returns content from your knowledge sources without generative alteration. This is recommended for interaction with Foundry Agent Service

In [5]:
from azure.search.documents.indexes.models import KnowledgeBase, KnowledgeSourceReference, AzureOpenAIVectorizerParameters, KnowledgeRetrievalOutputMode, KnowledgeRetrievalMinimalReasoningEffort
from azure.search.documents.indexes import SearchIndexClient

knowledge_base = KnowledgeBase(
    name=base_name,
    knowledge_sources=[
        KnowledgeSourceReference(
            name=knowledge_source_name
        )
    ],
    output_mode=KnowledgeRetrievalOutputMode.EXTRACTIVE_DATA,
    retrieval_reasoning_effort=KnowledgeRetrievalMinimalReasoningEffort()
)


index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_knowledge_base(knowledge_base=knowledge_base)
print(f"Knowledge base '{base_name}' created or updated successfully")

mcp_endpoint = f"{endpoint}/knowledgebases/{base_name}/mcp?api-version=2025-11-01-Preview"

Knowledge base 'earth-knowledge-base' created or updated successfully


## Create an agent

In Foundry Agent Service, an agent is a smart micro-service that can do RAG. The purpose of this agent is to decide when to send a query to the agentic retrieval pipeline.

In [ ]:
from azure.ai.projects import AIProjectClient

project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

list(project_client.agents.list())

## Create an MCP tool connection

In Microsoft Foundry, you must create a connection to authenticate to your MCP tool.

In [ ]:
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.cognitiveservices.models import ConnectionPropertiesV2BasicResource, CustomKeysConnectionProperties, CustomKeys
import requests
from azure.identity import get_bearer_token_provider

# Requires the Foundry project to have Search Index Data Reader role on the search service
bearer_token_provider = get_bearer_token_provider(credential, "https://management.azure.com/.default")
headers = {
    "Authorization": f"Bearer {bearer_token_provider()}",
}
response = requests.put(
    f"https://management.azure.com{project_resource_id}/connections/{project_connection_name}?api-version=2025-10-01-preview",
    headers=headers,
    json={
        "name": project_connection_name,
        "type": "Microsoft.MachineLearningServices/workspaces/connections",
        "properties": {
            "authType": "ProjectManagedIdentity",
            "category": "RemoteTool",
            "target": mcp_endpoint,
            "isSharedToAll": True,
            "audience": "https://search.azure.com/",
            "metadata": { "ApiType": "Azure" }
        }
    }
)
response.raise_for_status()
print(f"Connection '{project_connection_name}' created or updated successfully.")

Connection 'earthknowledgeconnection' created or updated successfully.


In [8]:
from azure.ai.projects.models import PromptAgentDefinition, MCPTool

instructions = """
A Q&A agent that can answer questions about the Earth at night.
Always provide references to the ID of the data source used to answer the question.
If you do not have the answer, respond with "I don't know".
"""
mcp_kb_tool = MCPTool(
    server_label="knowledge-base",
    server_url=mcp_endpoint,
    require_approval="never",
    allowed_tools=["knowledge_base_retrieve"],
    project_connection_id=project_connection_name
)
agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=agent_model,
        instructions=instructions,
        tools=[mcp_kb_tool]
    )
)


print(f"AI agent '{agent_name}' created or updated successfully")

AI agent 'earth-knowledge-agent' created or updated successfully


## Start a chat with the agent

In [9]:
# Get the OpenAI client for responses and conversations
openai_client = project_client.get_openai_client()

conversation = openai_client.conversations.create()

# Send initial request that will trigger the MCP tool
response = openai_client.responses.create(
    conversation=conversation.id,
    input="""
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print(f"Response: {response.output_text}")


Response: The larger December brightening observed in suburban belts compared to urban cores, despite higher absolute light levels downtown, is primarily because suburban areas have more yard space and a prevalence of single-family homes where holiday light displays are more common and prominent. Urban cores do experience brightening during the holidays, but suburban areas show greater increases in lighting intensity due to more extensive holiday light decorations in residential yards and neighborhoods [source 4:4, page 176].

Regarding the sharp visibility of Phoenix's nighttime street grid from space, the Phoenix metropolitan area is laid out along a regular grid of city blocks and streets, which is clearly visible at night due to street lighting. The sharp street grid pattern is emphasized by the extensive street lighting that traces the grid, and the presence of brightly lit transportation corridors (like Grand Avenue) and nodes such as shopping centers and gas stations at street i

## (Optional) Add remote SharePoint as a knowledge source

Adding a remote SharePoint knowledge source requires an additional `x-ms-query-source-authorization` header in your MCP connection.

In [10]:
from azure.search.documents.indexes.models import RemoteSharePointKnowledgeSource, KnowledgeSourceReference
from azure.search.documents.indexes import SearchIndexClient
from azure.identity import get_bearer_token_provider

remote_sp_ks = RemoteSharePointKnowledgeSource(
    name="remote-sharepoint",
    description="SharePoint knowledge source"
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_knowledge_source(knowledge_source=remote_sp_ks)
print(f"Knowledge source '{remote_sp_ks.name}' created or updated successfully.")

knowledge_base.knowledge_sources = [
    KnowledgeSourceReference(name=remote_sp_ks.name), KnowledgeSourceReference(name=knowledge_source_name)
]
index_client.create_or_update_knowledge_base(knowledge_base=knowledge_base)
print(f"Knowledge base '{base_name}' updated with new knowledge source successfully")

mcp_kb_tool = MCPTool(
    server_label="knowledge-base",
    server_url=mcp_endpoint,
    require_approval="never",
    allowed_tools=["knowledge_base_retrieve"],
    project_connection_id=project_connection_name,
    headers={
        "x-ms-query-source-authorization": get_bearer_token_provider(credential, "https://search.azure.com/.default")()
    }
)
agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=agent_model,
        instructions=instructions,
        tools=[mcp_kb_tool]
    )
)


print(f"AI agent '{agent_name}' created or updated successfully")

Knowledge source 'remote-sharepoint' created or updated successfully.
Knowledge base 'earth-knowledge-base' updated with new knowledge source successfully
AI agent 'earth-knowledge-agent' created or updated successfully


## Clean up objects and resources

If you no longer need Azure AI Search or Microsoft Foundry, delete the resources from your Azure subscription. You can also start over by deleting individual objects.

### Delete the agent

In [11]:
project_client.agents.delete_version(agent.name, agent.version)
print(f"AI agent '{agent.name}' version '{agent.version}' deleted successfully")

AI agent 'earth-knowledge-agent' version '4' deleted successfully


### Delete the knowledge base

In [12]:
index_client.delete_knowledge_base(base_name)
print(f"Knowledge base '{base_name}' deleted successfully")

Knowledge base 'earth-knowledge-base' deleted successfully


### Delete the knowledge source

In [13]:
index_client.delete_knowledge_source(knowledge_source=knowledge_source_name) # This is new feature in 2025-08-01-Preview api version
print(f"Knowledge source '{knowledge_source_name}' deleted successfully.")


Knowledge source 'earth-knowledge-source' deleted successfully.


### Delete the search index

In [14]:
index_client.delete_index(index)
print(f"Index '{index_name}' deleted successfully")

Index 'earth-at-night' deleted successfully
